In [1]:
from menagerie.sparse_interpretability.config import TrainConfig

train_config = TrainConfig()

In [2]:
import random

import numpy as np
import torch

torch.manual_seed(train_config.seed)
torch.cuda.manual_seed(train_config.seed)
np.random.seed(train_config.seed)
random.seed(train_config.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
from huggingface_hub import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

BASE_MODEL_ID = "openai-community/gpt2"

snapshot_download(BASE_MODEL_ID, allow_patterns=["*.safetensors", "*.json"])

transformer_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [4]:
from menagerie.sparse_interpretability.dataset import (
    compute_activations,
    tokenize_and_concat,
)

from datasets import IterableDataset, load_dataset

# Assuming the tokenizer and data loading are the same as before
data = load_dataset("NeelNanda/c4-code-20k", split="train", streaming=True)
assert isinstance(data, IterableDataset), f"Expected Dataset, got {type(data)}"

tokenized_data = tokenize_and_concat(
    data,
    tokenizer,  # type: ignore
    max_length=train_config.seq_len,
)
tokenized_data = tokenized_data.shuffle(seed=train_config.seed)
activations_data = compute_activations(
    tokenized_data,
    transformer_model,
    train_config.activation_layer_name,
    train_config.activation_size,
)

In [22]:
dataset = activations_data.filter(
    lambda x: x["tokens"].shape == torch.Size([train_config.seq_len])
    and x["activations"].shape == torch.Size([train_config.seq_len, 768])
).take(325000)

In [24]:
# Iterable datasets do not have a native way to stream upload to the huggingface hub
# so as a hack you can stream the mapping to disk, only keeping a slice of the data in
# temporary files, and then upload the temporary files to the hub

import tempfile

from datasets import Dataset

with tempfile.TemporaryDirectory() as tmpdirname:
    ds_on_disk = Dataset.from_generator(
        dataset.__iter__, cache_dir=tmpdirname
    )  # stream to 'disk'
    assert isinstance(ds_on_disk, Dataset), f"Expected Dataset, got {type(ds_on_disk)}"

    dataset_name = "walln/c4-code-20k-gpt-2-activations"
    dataset_subset_name = train_config.activation_layer_name
    ds_on_disk.push_to_hub(dataset_name, dataset_subset_name)

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/163 [00:00<?, ?it/s]

Uploading the dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/27.0 [00:00<?, ?B/s]